In [1]:
import pandas as pd
import numpy as np

# ==============================================================================
# 1. EXTRACT & SETUP
# ==============================================================================
print("[ 1. EXTRACT DATA ]")
df = pd.read_csv('train.csv')

print(f"Initial Shape : {df.shape} (Rows, Cols)")
print("Columns       :", list(df.columns))

# ==============================================================================
# 2. DATA CLEANING & PREPROCESSING
# ==============================================================================
print("\n[ 2. CLEANING ]")
print("\n--- BEFORE CLEANING ---")
print(f"Shape: {df.shape}")
missing_before = df.isnull().sum()
print(f"Missing Values:\n{missing_before[missing_before > 0]}")
print("First 3 rows:")
print(df.head(3))

# Convert date columns to datetime objects
df['Order Date'] = pd.to_datetime(df['Order Date'], dayfirst=True, errors='coerce')
df['Ship Date']  = pd.to_datetime(df['Ship Date'], dayfirst=True, errors='coerce')

# Handle Missing Values (Postal Code)
postal_mean = df['Postal Code'].mean()
df['Postal Code'] = df['Postal Code'].fillna(postal_mean).astype(int).astype(str)

print("\n--- AFTER CLEANING ---")
print(f"Shape: {df.shape}")
missing_after = df.isnull().sum()
print(f"Missing Values:\n{missing_after[missing_after > 0]}")  # Should be empty
print("First 3 rows (Cleaned):")
print(df.head(3))
print("✔ Dates converted & Missing values handled")

# ==============================================================================
# 3. FEATURE ENGINEERING (ALASAN 18 KOLOM -> 20 KOLOM)
# ==============================================================================
print("\n[ 3. CREATING KEYS (18 -> 20 Cols) ]")
print("Menambahkan 2 kolom helper (key) untuk kebutuhan join Star Schema:")
print("  1. region_key: Gabungan Region + State + City")
print("  2. product_key: Gabungan Product ID + Product Name")

# Membuat Unique Key untuk Region
df['region_key'] = (
    df['Region'].astype(str) + '-' +
    df['State'].astype(str) + '-' +
    df['City'].astype(str)
)

# Membuat Unique Key untuk Product
df['product_key'] = (
    df['Product ID'].astype(str) + '-' +
    df['Product Name'].astype(str)
)

print(f"New Shape     : {df.shape} (Total 20 Kolom sekarang)")

# ==============================================================================
# 4. CREATE DIMENSION TABLES
# ==============================================================================
print("\n[ 4. GENERATING DIMENSIONS ]")

# --- DIM_DATE ---
date_range = pd.date_range(
    start=df[['Order Date','Ship Date']].min().min(),
    end=df[['Order Date','Ship Date']].max().max()
)
dim_date = pd.DataFrame({'date': date_range})
dim_date['sk_date']     = dim_date['date'].dt.strftime('%Y%m%d').astype(int)
dim_date['year']        = dim_date['date'].dt.year
dim_date['month_num']   = dim_date['date'].dt.month
dim_date['month_name']  = dim_date['date'].dt.strftime('%B')
dim_date['quarter']     = dim_date['date'].dt.quarter
dim_date['day_of_week'] = dim_date['date'].dt.strftime('%A')
dim_date = dim_date[['sk_date', 'date', 'year', 'month_num', 'month_name', 'quarter', 'day_of_week']]

# --- DIM_CUSTOMER ---
dim_customer = df[['Customer ID', 'Customer Name', 'Segment']].drop_duplicates().reset_index(drop=True)
dim_customer['sk_customer'] = dim_customer.index + 1
dim_customer = dim_customer[['sk_customer', 'Customer ID', 'Customer Name', 'Segment']]
dim_customer.columns = ['sk_customer', 'customer_id', 'customer_name', 'segment']

# --- DIM_PRODUCT ---
dim_product = df[['Product ID', 'Product Name', 'Category', 'Sub-Category', 'product_key']].drop_duplicates(subset=['product_key']).reset_index(drop=True)
dim_product['sk_product'] = dim_product.index + 1
dim_product = dim_product[['sk_product', 'Product ID', 'Product Name', 'Category', 'Sub-Category', 'product_key']]
dim_product.columns = ['sk_product', 'product_id', 'product_name', 'category', 'sub_category', 'product_key']

# --- DIM_REGION ---
dim_region = df[['Region', 'Country', 'State', 'City', 'region_key']].drop_duplicates(subset=['region_key']).reset_index(drop=True)
dim_region['sk_region'] = dim_region.index + 1
dim_region = dim_region[['sk_region', 'Region', 'Country', 'State', 'City', 'region_key']]
dim_region.columns = ['sk_region', 'region', 'country', 'state', 'city', 'region_key']

print(f"✔ Dimensions Created: Date({len(dim_date)}), Customer({len(dim_customer)}), Product({len(dim_product)}), Region({len(dim_region)})")

# ==============================================================================
# 5. CREATE FACT TABLE
# ==============================================================================
print("\n[ 5. GENERATING FACT TABLE ]")
fact_sales = df.copy()

# Merge with Dimensions to get Surrogate Keys (SK)
fact_sales = fact_sales.merge(dim_date[['sk_date', 'date']], left_on='Order Date', right_on='date', how='left').rename(columns={'sk_date': 'sk_order_date'}).drop(columns=['date'])
fact_sales = fact_sales.merge(dim_date[['sk_date', 'date']], left_on='Ship Date', right_on='date', how='left').rename(columns={'sk_date': 'sk_ship_date'}).drop(columns=['date'])
fact_sales = fact_sales.merge(dim_customer[['sk_customer', 'customer_id']], left_on='Customer ID', right_on='customer_id', how='left').drop(columns=['customer_id'])
fact_sales = fact_sales.merge(dim_product[['sk_product', 'product_key']], on='product_key', how='left')
fact_sales = fact_sales.merge(dim_region[['sk_region', 'region_key']], on='region_key', how='left')

# Select only Fact Columns (Include Profit & Quantity as per Rubric)
# Check if Profit/Quantity exists (Standard Superstore), otherwise handle gracefully
cols_to_keep = ['Order ID', 'sk_order_date', 'sk_ship_date', 'sk_customer', 'sk_product', 'sk_region', 'Sales']
if 'Profit' in df.columns:
    cols_to_keep.append('Profit')
if 'Quantity' in df.columns:
    cols_to_keep.append('Quantity')
if 'Discount' in df.columns:
    cols_to_keep.append('Discount')

fact_sales = fact_sales[cols_to_keep]

# Rename columns to snake_case
col_map = {
    'Order ID': 'order_id',
    'Order Date': 'order_date',
    'Sales': 'sales',
    'Profit': 'profit',
    'Quantity': 'quantity',
    'Discount': 'discount'
}
fact_sales.rename(columns=col_map, inplace=True)

print(f"✔ Fact Table Created: {fact_sales.shape}")
print(f"  Columns: {list(fact_sales.columns)}")

# ==============================================================================
# 6. VALIDATION & SAVING
# ==============================================================================
print("\n[ 6. VALIDATION & SAVING ]")
missing_sks = fact_sales[['sk_customer', 'sk_product', 'sk_region', 'sk_order_date', 'sk_ship_date']].isna().sum().sum()
print(f"Total Missing SKs : {missing_sks} (Should be 0)")

# Save files
dim_date.to_csv('dim_date.csv', index=False)
dim_customer.to_csv('dim_customer.csv', index=False)
dim_product.to_csv('dim_product.csv', index=False)
dim_region.to_csv('dim_region.csv', index=False)
fact_sales.to_csv('fact_sales.csv', index=False)

print("✔ All files saved successfully.")
print("ETL PROCES COMPLETED.")

[ 1. EXTRACT DATA ]
Initial Shape : (9800, 18) (Rows, Cols)
Columns       : ['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode', 'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State', 'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category', 'Product Name', 'Sales']

[ 2. CLEANING ]

--- BEFORE CLEANING ---
Shape: (9800, 18)
Missing Values:
Postal Code    11
dtype: int64
First 3 rows:
   Row ID        Order ID  Order Date   Ship Date     Ship Mode Customer ID  \
0       1  CA-2017-152156  08/11/2017  11/11/2017  Second Class    CG-12520   
1       2  CA-2017-152156  08/11/2017  11/11/2017  Second Class    CG-12520   
2       3  CA-2017-138688  12/06/2017  16/06/2017  Second Class    DV-13045   

     Customer Name    Segment        Country         City       State  \
0      Claire Gute   Consumer  United States    Henderson    Kentucky   
1      Claire Gute   Consumer  United States    Henderson    Kentucky   
2  Darrin Van Huff  Corporate  United Sta